# 빅데이터 분석기사 실기 준비 놀이터
- https://www.kaggle.com/datasets/agileteam/bigdatacertificationkr

## T2-6. Bike-sharing-demand (Regression) / 자전거 수요 예측

### 6. 자전거 수요 예측
- 데이터셋 : Bike Sharing Demand_train.csv

#### 0. 시험 환경 세팅

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./data/Bike Sharing Demand_train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='count')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8708, 12), (2178, 12), (8708, 2), (2178, 2))

#### 1. 라이브러리 및 데이터 호출

In [2]:
import numpy as np
import pandas as pd

X_train.drop('id', axis=1, inplace=True)
id = X_test.pop('id')
y = y_train['count']

X_train.shape, X_test.shape, y.shape

((8708, 11), (2178, 11), (8708,))

#### 2. EDA

In [3]:
display(X_train.info(), X_train.head(), X_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8708 entries, 9827 to 9332
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    8708 non-null   object 
 1   season      8708 non-null   int64  
 2   holiday     8708 non-null   int64  
 3   workingday  8708 non-null   int64  
 4   weather     8708 non-null   int64  
 5   temp        8708 non-null   float64
 6   atemp       8708 non-null   float64
 7   humidity    8708 non-null   int64  
 8   windspeed   8708 non-null   float64
 9   casual      8708 non-null   int64  
 10  registered  8708 non-null   int64  
dtypes: float64(3), int64(7), object(1)
memory usage: 816.4+ KB


None

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered
9827,2012-10-13 20:00:00,4,0,0,1,16.40,20.455,54,6.0032,57,198
3403,2011-08-11 17:00:00,3,0,1,1,32.80,34.850,29,16.9979,96,486
4960,2011-11-19 18:00:00,4,0,0,1,15.58,19.695,46,8.9981,36,172
4399,2011-10-15 08:00:00,4,0,0,1,18.86,22.725,59,19.0012,24,101
3999,2011-09-17 16:00:00,3,0,0,1,22.14,25.760,68,16.9979,116,189


datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
dtype: int64

In [4]:
display(X_test.info(), X_test.head(), X_test.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2178 entries, 8705 to 10249
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    2178 non-null   object 
 1   season      2178 non-null   int64  
 2   holiday     2178 non-null   int64  
 3   workingday  2178 non-null   int64  
 4   weather     2178 non-null   int64  
 5   temp        2178 non-null   float64
 6   atemp       2178 non-null   float64
 7   humidity    2178 non-null   int64  
 8   windspeed   2178 non-null   float64
 9   casual      2178 non-null   int64  
 10  registered  2178 non-null   int64  
dtypes: float64(3), int64(7), object(1)
memory usage: 204.2+ KB


None

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered
8705,2012-08-05 02:00:00,3,0,0,1,30.34,34.850,70,19.9995,5,68
3262,2011-08-05 20:00:00,3,0,1,1,29.52,33.335,58,15.0013,70,221
5193,2011-12-10 11:00:00,4,0,0,1,13.12,14.395,53,27.9993,44,192
8969,2012-08-16 02:00:00,3,0,1,1,26.24,30.305,69,12.9980,0,15
7614,2012-05-16 15:00:00,2,0,1,2,29.52,33.335,48,7.0015,52,222


datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
dtype: int64

In [5]:
for c in X_train.columns:
    print(c, X_train[c].nunique())
    print(np.sort(X_train[c].unique()))
    print('='*100)

datetime 8708
['2011-01-01 00:00:00' '2011-01-01 02:00:00' '2011-01-01 03:00:00' ...
 '2012-12-19 21:00:00' '2012-12-19 22:00:00' '2012-12-19 23:00:00']
season 4
[1 2 3 4]
holiday 2
[0 1]
workingday 2
[0 1]
weather 4
[1 2 3 4]
temp 48
[ 0.82  1.64  2.46  3.28  4.1   4.92  5.74  6.56  7.38  8.2   9.02  9.84
 10.66 11.48 12.3  13.12 13.94 14.76 15.58 16.4  17.22 18.04 18.86 19.68
 20.5  21.32 22.14 22.96 23.78 24.6  25.42 26.24 27.06 27.88 28.7  29.52
 30.34 31.16 31.98 32.8  33.62 34.44 35.26 36.08 36.9  37.72 38.54 39.36]
atemp 60
[ 0.76   1.515  2.275  3.03   3.79   4.545  5.305  6.06   6.82   7.575
  8.335  9.09   9.85  10.605 11.365 12.12  12.88  13.635 14.395 15.15
 15.91  16.665 17.425 18.18  18.94  19.695 20.455 21.21  21.97  22.725
 23.485 24.24  25.    25.76  26.515 27.275 28.03  28.79  29.545 30.305
 31.06  31.82  32.575 33.335 34.09  34.85  35.605 36.365 37.12  37.88
 38.635 39.395 40.15  40.91  41.665 42.425 43.18  43.94  44.695 45.455]
humidity 88
[  0   8  10  12  13  15  

#### 3. Preprocessing
- datetime 컬럼 타입 변경 -> 제거
- 표준화(MInMaxScaler)

> datetime 컬럼 타입 변경(보류) -> 제거

In [6]:
# X_train['datetime'] = pd.to_datetime(X_train['datetime'])
# X_test['datetime'] = pd.to_datetime(X_test['datetime'])
# print(X_train['datetime'].dtypes, X_test['datetime'].dtypes)

In [7]:
X_train = X_train.drop('datetime', axis=1)
X_test = X_test.drop('datetime', axis=1)

X_train.shape, X_test.shape

((8708, 10), (2178, 10))

> 표준화

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### 4. Modeling

In [9]:
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor, XGBRFRegressor

from sklearn.metrics import mean_squared_error, r2_score

In [10]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size=0.2, shuffle=True, random_state=156)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((6966, 10), (1742, 10), (6966,), (1742,))

In [11]:
rf = RandomForestRegressor(random_state=156)
gb = GradientBoostingRegressor(random_state=156)
xgb = XGBRegressor(random_state=156)
xgbrf = XGBRFRegressor(random_state=156)

models = [rf, gb, xgb, xgbrf]

In [12]:
for model in models:
    model.fit(X_tr, y_tr)
    pred = model.predict(X_val)
    
    name = str(model).split('(')[0]
    print(name)
    print('rmse :', np.sqrt(mean_squared_error(y_val, pred)))
    print('r2 :', r2_score(y_val, pred))

RandomForestRegressor
rmse : 2.622185134472565
r2 : 0.999802751645575
GradientBoostingRegressor
rmse : 5.110253648400705
r2 : 0.9992508451410245
XGBRegressor
rmse : 3.726797555090076
r2 : 0.9996015642576978
XGBRFRegressor
rmse : 8.414178083440722
r2 : 0.9979689992408952


In [13]:
rf = RandomForestRegressor(random_state=156)

rf.fit(X_train, y)
pred = rf.predict(X_test)

In [14]:
out_data = pd.DataFrame({'id': id, 'count': pred})
out_data

,id,count
8705,8705,72.71
3262,3262,289.57
5193,5193,236.24
8969,8969,15.02
7614,7614,274.86
...,...,...
7936,7936,23.13
4583,4583,17.03
1279,1279,5.00
8195,8195,396.00


In [15]:
# out_data.to_csv('123456.csv', index=False)

In [16]:
# 가채점
print('rmse :', np.sqrt(mean_squared_error(y_test['count'], pred)))
print('r2 :', r2_score(y_test['count'], pred))

rmse : 2.595475074659209
r2 : 0.9997882573697187


#### 5. 다른 모델 평가

In [17]:
for model in models:
    model.fit(X_train, y)
    pred = model.predict(X_test)
    
    name = str(model).split('(')[0]
    print(name)
    print('RMSE :', np.sqrt(mean_squared_error(y_test['count'], pred)))
    print('r2 :', r2_score(y_test['count'], pred))
    print('='*100)

RandomForestRegressor
RMSE : 2.595475074659209
r2 : 0.9997882573697187
GradientBoostingRegressor
RMSE : 5.55425575126943
r2 : 0.9990303247175842
XGBRegressor
RMSE : 3.2742415576245394
r2 : 0.9996630262962873
XGBRFRegressor
RMSE : 8.9373462015789
r2 : 0.9974893178139282
